In [3]:
import pandas as pd
from pandas import Series, DataFrame
import datetime as dt
import time

In [149]:
def most_delayed_flights(filename):
    
    flight_delays_df = pd.read_csv(filename)
    
    flight_delays_df['FL_DATE'] = pd.to_timedelta(flight_delays_df['FL_DATE'], unit = 'D') + dt.datetime(1899, 12, 30)
    flight_delays_df.set_index(['FL_DATE'], inplace = True)
    
    carrier_delays = flight_delays_df.groupby(['CARRIER', 'Flight Status']).size().unstack().fillna(0)
    carrier_delays['avg'] = carrier_delays['delayed'] / (carrier_delays['delayed'] + carrier_delays['ontime'])
    carrier_delays.reset_index(inplace = True)
    
    flights_df = flight_delays_df.groupby([pd.Grouper(freq = '7D'),'CARRIER', 'FL_NUM', 'Flight Status']).size().unstack().fillna(0)
    flights_df['total flights'] = flights_df['delayed'] + flights_df['ontime']
    
    flights_df = flights_df.where(flights_df['total flights'] >= 3).dropna().reset_index()
    
    flights_df['mean'] = flights_df['delayed'] / flights_df['total flights']
    merged_df = flights_df.merge(carrier_delays, left_on = 'CARRIER', right_on = 'CARRIER', how = 'inner')

    final_df = merged_df[merged_df['mean'] > merged_df['avg']]
    return final_df.groupby(['CARRIER', 'FL_NUM'])['mean'].sum().rename(index = None)
    

In [150]:
most_delayed_flights('flight_delays.csv')

CARRIER  FL_NUM
CO       814       0.600000
DH       7211      0.600000
         7215      0.428571
         7302      0.333333
         7303      0.714286
         7304      0.333333
         7307      0.600000
         7812      0.285714
         7814      0.571429
DL       746       0.142857
         1766      0.333333
MQ       4752      0.285714
         4784      0.333333
         4968      0.600000
         4970      0.666667
         4976      0.600000
RU       2156      0.571429
         2261      0.400000
         2303      0.428571
         2336      0.500000
         2385      0.600000
         2403      0.333333
US       1479      0.333333
         2176      0.142857
         2178      0.250000
         2186      0.250000
dtype: float64

In [8]:
flight_delays_df = pd.read_csv('flight_delays.csv')
        
flight_delays_df['FL_DATE'] = pd.to_timedelta(flight_delays_df['FL_DATE'], unit = 'D') + dt.datetime(1899, 12, 30)
flight_delays_df.set_index(['FL_DATE'], inplace = True)

In [112]:
flight_delays_df.head(1)

,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,TAIL_NUM,Flight Status
FL_DATE,,,,,,,,,,,,
2004-01-01,1455,OH,1455,JFK,184,5935,BWI,0,4,1,N940CA,ontime


In [23]:
test_1_df = flight_delays_df.groupby(['CARRIER', 'FL_NUM']).resample('W-Sun').count()

In [55]:
#pd.Grouper(key='Date', freq='W-MON')
carrier_delays = flight_delays_df.groupby(['CARRIER', 'Flight Status']).size().unstack().fillna(0)

In [63]:
flight_delays_df.groupby(['CARRIER', 'Flight Status']).size().unstack().fillna(0)

Flight Status,delayed,ontime
CARRIER,,
CO,7.0,12.0
DH,32.0,93.0
DL,2.0,72.0
MQ,12.0,52.0
OH,2.0,4.0
RU,28.0,67.0
UA,0.0,7.0
US,4.0,78.0


In [65]:
carrier_delays['avg'] = carrier_delays['delayed'] / (carrier_delays['delayed'] + carrier_delays['ontime'])

In [83]:
carrier_delays.reset_index(inplace = True)

In [106]:
testdf1 = flight_delays_df.groupby([pd.Grouper(freq = 'W-Sun'), 'CARRIER', 'FL_NUM', 'Flight Status']).size().unstack().fillna(0).reset_index()

In [107]:
testdf1[testdf1['FL_NUM'] == 814]

Flight Status,FL_DATE,CARRIER,FL_NUM,delayed,ontime
3,2004-01-04,CO,814,0.0,2.0
84,2004-01-11,CO,814,3.0,0.0


In [134]:
#flight_delays_df.resample('7D').count()

,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,TAIL_NUM,Flight Status
FL_DATE,,,,,,,,,,,,
2004-01-01,472,472,472,472,472,472,472,472,472,472,472,472


In [93]:
pd.crosstab([flight_delays_df.CARRIER, flight_delays_df.FL_NUM] , flight_delays_df['Flight Status']).reset_index()

Flight Status,CARRIER,FL_NUM,delayed,ontime
0,CO,806,0,4
1,CO,808,1,6
2,CO,810,1,0
3,CO,814,3,2
4,CO,816,2,0
5,DH,6155,1,4
6,DH,7208,1,6
7,DH,7211,3,2
8,DH,7215,3,4
9,DH,7299,1,6


In [135]:
flights_df = flight_delays_df.groupby([pd.Grouper(freq = '7D'),'CARRIER', 'FL_NUM', 'Flight Status']).size().unstack().fillna(0)

In [136]:
flights_df['total flights'] = flights_df['delayed'] + flights_df['ontime']

In [137]:
flights_df = flights_df.where(flights_df['total flights'] >= 3).dropna().reset_index()

CARRIER  FL_NUM
CO       814       0.600000
DH       7211      0.600000
         7215      0.428571
         7302      0.333333
         7303      0.714286
         7304      0.333333
         7307      0.600000
         7812      0.285714
         7814      0.571429
DL       746       0.142857
         1766      0.333333
MQ       4752      0.285714
         4784      0.333333
         4968      0.600000
         4970      0.666667
         4976      0.600000
RU       2156      0.571429
         2261      0.400000
         2303      0.428571
         2336      0.500000
         2385      0.600000
         2403      0.333333
US       1479      0.333333
         2176      0.142857
         2178      0.250000
         2186      0.250000
dtype: float64